# Code to generate folder for HMC run
May 8, 2023

In [5]:
import numpy as np
import os
import argparse
import subprocess as sp
import shutil
import glob

import yaml

In [6]:
def f_write_config(input_dict,fname):
    
    with open(fname,'w') as f:
        yaml.dump(input_dict,f,sort_keys=False)
        
# f_write_config(dict_pars,'config.yaml')

In [7]:
def f_build_submit_script(dict_pars,fname):
    
    submit_strg='''#!/bin/bash
################
# Begin LSF Directives
#SBATCH -A latticgc
#SBATCH -t 00:10:00
#SBATCH -J hsdm
#SBATCH -o hsdm.%J
#SBATCH -e hsdm.%J
#SBATCH -N {N}
#SBATCH -n {nprocs}
#SBATCH --exclusive
#SBATCH --gpu-bind=map_gpu:0,1,2,3,7,6,5,4
#SBATCH -c 1
###SBATCH --threads-per-core=1

echo "--start " `date` `date +%s`

export BIND="--cpu-bind=verbose,map_ldom:3,3,1,1,2,2,0,0"

export MPIR_CVAR_GPU_EAGER_DEVICE_MEM=16384
export MPICH_GPU_SUPPORT_ENABLED=1
export MPICH_SMP_SINGLE_COPY_MODE=CMA
export OMP_NUM_THREADS=8
export MPICH_OFI_NIC_POLICY=GPU
export OPT="--comms-concurrent --comms-overlap "

source $GRID_DIR/setup_env.sh
export TSAN_OPTIONS='ignore_noninstrumented_modules=1'
export LD_LIBRARY_PATH

Ls={dwf_Ls}
traj_l={traj_l}
md_steps={md_steps}
BETA={beta}
M_F={mf}

APP="$RUN_DIR/dm_tests/build/dweofa_mobius_HSDM --grid {Lx}.{Lx}.{Lx}.{Lt} --mpi {mpi} --shm 2048 --shm-force-mpi 1 --device-mem 5000 --Trajectories 200 --Thermalizations 10 $OPT $Ls $traj_l $md_steps $BETA $M_F"
#srun --gpus-per-task 1 -n64 $BIND $APP > HSDM.out
#srun --gpus-per-task 1 -n64 $APP > HSDM.out
srun -n{nprocs} $APP > HSDM.out

echo "--end " `date` `date +%s`

    '''.format(**dict_pars)
    with open(fname,'w') as f: f.write(submit_strg)


In [8]:
run_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/'

In [13]:
Lx=16; Lt=8
N=1
mx,my,mz,mt=2,2,2,1
Ls=16
dict_pars={
    'Lx':Lx, 'Lt':Lt, # Lattice size 
    'F_action': 'Mobius_dwf',
    
    'traj_l':1, 'md_steps':10, 
#     'beta':beta, 'mf':mf, 
    'dwf_Ls':Ls, 
    'mpi':".".join([str(i) for i in [mx,my,mz,mt]]),
    'nprocs': mx*my*mz*mt,
    'N':N  ## Number of nodes
          }

In [14]:
print(dict_pars)

{'Lx': 16, 'Lt': 8, 'F_action': 'Mobius_dwf', 'traj_l': 1, 'md_steps': 10, 'dwf_Ls': 16, 'mpi': '2.2.2.1', 'nprocs': 8, 'N': 1}


In [15]:
beta_list=[10.0,10.8,12.0]
m_list=[0.1]

for beta in beta_list:
    for mf in m_list:
        dict_pars['beta']=beta
        dict_pars['mf']=mf

        strg='run_Lx-%s_Lt-%s_Ls-%s_beta-%s_mf-%s'%(Lx,Lt,Ls,beta,mf)
        fldr=run_dir+'%s/'%(strg)

        print(fldr)

        if os.path.exists(fldr): 
            print("Error: Directory %s exists",fldr)
            raise SystemError
        else:
            os.makedirs(run_dir+'/{0}'.format(strg))
        #     shutil.copy(src,fldr)
            fname=fldr+'submit_2_dwf-hsdm.sh'
            f_build_submit_script(dict_pars,fname)

            fname=fldr+'config.yaml'
            f_write_config(dict_pars,fname)



/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/run_Lx-16_Lt-8_Ls-16_beta-10.0_mf-0.1/
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/run_Lx-16_Lt-8_Ls-16_beta-10.8_mf-0.1/
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/run_Lx-16_Lt-8_Ls-16_beta-12.0_mf-0.1/


### Code to rename folders
test_dir=run_dir+'../new_runs/test/'
os.chdir(test_dir)
fldr_lst=glob.glob('run_L*')
print(fldr_lst)
for src in fldr_lst:
     target=src.replace('run_Ls','run_Lx')
     print(src,target)
     os.rename(src,target)

In [12]:
! ls /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/

old_runs_small_beta
run_Lx-16_Lt-32_Ls-16_beta-10.8_mf-0.02
run_Lx-16_Lt-32_Ls-16_beta-10.8_mf-0.1
run_Lx-16_Lt-32_Ls-32_beta-10.8_mf-0.02
run_Lx-16_Lt-32_Ls-32_beta-10.8_mf-0.1
run_Lx-16_Lt-32_Ls-8_beta-10.8_mf-0.1
